# Boolean Functions as Random Variables

**Reference: O'Donnell, Chapters 1-3**

This notebook explores the probabilistic view of Boolean functions, demonstrating how to:

1. View Boolean functions as random variables on the hypercube
2. Sample from uniform and p-biased distributions
3. Sample from the Fourier (spectral) distribution
4. Estimate Fourier coefficients via Monte Carlo
5. Verify convergence and the law of large numbers

---

## Key Concepts (from O'Donnell)

- A Boolean function $f: \{-1,+1\}^n \to \{-1,+1\}$ is a **random variable** when inputs are drawn uniformly
- $\mathbb{E}[f] = \hat{f}(\emptyset)$ (the Fourier coefficient on the empty set)
- $\text{Var}[f] = \sum_{S \neq \emptyset} \hat{f}(S)^2$ by Parseval
- The **spectral distribution** has $\Pr[S] = \hat{f}(S)^2$ (for $\pm 1$-valued $f$)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import boofun as bf

# Import the sampling module
from boofun.analysis.sampling import (
    sample_uniform,
    sample_biased,
    sample_spectral,
    sample_input_output_pairs,
    estimate_fourier_coefficient,
    estimate_influence,
    estimate_total_influence,
    RandomVariableView,
    SpectralDistribution,
)

# For reproducibility
np.random.seed(42)

print("boofun version:", bf.__version__)

## 1. Sampling from the Hypercube

### 1.1 Uniform Sampling

The most basic operation is sampling uniformly from $\{0,1\}^n$, equivalent to choosing each bit independently with probability $1/2$.

In [ ]:
# Sample 10 random inputs for n=4 variables
n = 4
samples = sample_uniform(n, n_samples=10)

print(f"10 uniform samples from {{0,1}}^{n}:")
for x in samples:
    bits = f"{x:0{n}b}"
    print(f"  {x:2d} = {bits}")

In [ ]:
# Verify uniformity: each input should appear roughly equally often
n = 3
samples = sample_uniform(n, n_samples=8000)
counts = np.bincount(samples, minlength=8)

plt.figure(figsize=(10, 4))
plt.bar(range(8), counts, color='steelblue', alpha=0.7)
plt.axhline(y=1000, color='red', linestyle='--', label=f'Expected = 1000')
plt.xlabel('Input x')
plt.ylabel('Count')
plt.title('Uniform Sampling: Each input appears ~1/8 of the time')
plt.xticks(range(8), [f"{x:03b}" for x in range(8)])
plt.legend()
plt.tight_layout()
plt.show()

### 1.2 P-Biased Sampling

The **$p$-biased distribution** $\mu_p$ sets each bit to 1 independently with probability $p$.

- $p = 0.5$: Uniform distribution
- $p < 0.5$: More zeros than ones
- $p > 0.5$: More ones than zeros

In [ ]:
# Compare p=0.2, p=0.5, p=0.8 distributions
n = 8
n_samples = 5000

def avg_hamming_weight(samples, n):
    """Average number of 1 bits."""
    return np.mean([bin(x).count('1') for x in samples])

for p in [0.2, 0.5, 0.8]:
    samples = sample_biased(n, p, n_samples)
    avg_ones = avg_hamming_weight(samples, n)
    expected = p * n
    print(f"p={p:.1f}: Average #ones = {avg_ones:.2f} (expected {expected:.2f})")

In [ ]:
# Visualize the distribution of Hamming weights
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

n = 8
for ax, p in zip(axes, [0.2, 0.5, 0.8]):
    samples = sample_biased(n, p, 10000)
    weights = [bin(x).count('1') for x in samples]
    
    ax.hist(weights, bins=range(n+2), alpha=0.7, color='steelblue', edgecolor='black')
    ax.axvline(x=p*n, color='red', linestyle='--', label=f'E[|x|] = {p*n}')
    ax.set_xlabel('Hamming weight')
    ax.set_ylabel('Count')
    ax.set_title(f'p = {p}')
    ax.legend()

plt.suptitle('Hamming Weight Distribution under μ_p')
plt.tight_layout()
plt.show()

## 2. Sampling Input-Output Pairs

Given a Boolean function $f$, we can sample $(x, f(x))$ pairs. This is the foundation of Monte Carlo estimation.

In [ ]:
# Sample from the AND function
f = bf.AND(3)
inputs, outputs = sample_input_output_pairs(f, n_samples=20)

print("Samples from AND₃:")
print(f"{'Input':<10} {'Binary':<10} {'f(x)':<5}")
print("-" * 25)
for x, y in zip(inputs[:10], outputs[:10]):
    print(f"{x:<10} {x:03b}        {y}")

# Estimate E[f] = Pr[AND(x) = 1] (should be 1/8 = 0.125)
print(f"\nEstimated Pr[AND₃ = 1] = {np.mean(outputs):.3f} (exact: 0.125)")

## 3. Spectral Sampling

**Spectral sampling** draws subsets $S$ with probability proportional to $\hat{f}(S)^2$.

For a $\pm 1$-valued function, Parseval says $\sum_S \hat{f}(S)^2 = 1$, so this is a valid distribution.

This is useful for:
- Finding **heavy Fourier coefficients**
- **Learning algorithms** (Goldreich-Levin)
- Understanding **spectral structure**

In [ ]:
# XOR has all Fourier weight on the full set {0,1}
xor = bf.create([0, 1, 1, 0])
samples = sample_spectral(xor, n_samples=100)

print("Spectral samples from XOR:")
unique, counts = np.unique(samples, return_counts=True)
for s, c in zip(unique, counts):
    bits = [i for i in range(2) if (s >> i) & 1]
    set_str = "{" + ",".join(map(str, bits)) + "}" if bits else "∅"
    print(f"  S = {set_str}: {c}% of samples")

print("\n→ XOR concentrates all spectral weight on S = {0,1} (the full set)")

In [ ]:
# AND has weight spread across multiple subsets
f = bf.AND(3)
samples = sample_spectral(f, n_samples=1000)

# Get exact Fourier coefficients for comparison
fourier = f.fourier()

print("Spectral distribution of AND₃:")
print(f"{'Subset':<12} {'f̂(S)²':<10} {'Sampled %':<10}")
print("-" * 35)

unique, counts = np.unique(samples, return_counts=True)
sampled_freq = dict(zip(unique, counts / 1000))

for s in range(8):
    bits = [i for i in range(3) if (s >> i) & 1]
    set_str = "{" + ",".join(map(str, bits)) + "}" if bits else "∅"
    exact = fourier[s]**2
    sampled = sampled_freq.get(s, 0)
    if exact > 0.001 or sampled > 0.001:
        print(f"{set_str:<12} {exact:<10.4f} {sampled*100:<10.1f}%")

## 4. Monte Carlo Estimation of Fourier Coefficients

The Fourier coefficient $\hat{f}(S)$ satisfies:

$$\hat{f}(S) = \mathbb{E}_x[f(x) \chi_S(x)]$$

where $\chi_S(x) = (-1)^{\langle x, S \rangle}$.

We can **estimate** this by sampling:

$$\hat{f}(S) \approx \frac{1}{N} \sum_{i=1}^{N} f(x_i) \chi_S(x_i)$$

In [ ]:
# Estimate Fourier coefficients of AND₃
f = bf.AND(3)
exact_fourier = f.fourier()

print("Monte Carlo estimation of AND₃ Fourier coefficients:")
print(f"{'Subset':<12} {'Exact f̂(S)':<12} {'Estimated':<12} {'Error':<10}")
print("-" * 50)

for s in range(8):
    bits = [i for i in range(3) if (s >> i) & 1]
    set_str = "{" + ",".join(map(str, bits)) + "}" if bits else "∅"
    
    exact = exact_fourier[s]
    est = estimate_fourier_coefficient(f, s, n_samples=10000)
    error = abs(exact - est)
    
    print(f"{set_str:<12} {exact:<12.4f} {est:<12.4f} {error:<10.4f}")

In [ ]:
# Show convergence with confidence intervals
f = bf.majority(3)
S = 1  # Coefficient on {0}
exact = f.fourier()[S]

sample_sizes = [100, 500, 1000, 5000, 10000]
estimates = []
errors = []

for n in sample_sizes:
    est, std_err = estimate_fourier_coefficient(f, S, n_samples=n, return_confidence=True)
    estimates.append(est)
    errors.append(1.96 * std_err)  # 95% CI

plt.figure(figsize=(10, 5))
plt.errorbar(sample_sizes, estimates, yerr=errors, fmt='o-', capsize=5, label='Estimate ± 95% CI')
plt.axhline(y=exact, color='red', linestyle='--', label=f'Exact: {exact:.4f}')
plt.xlabel('Number of samples')
plt.ylabel('Estimated f̂({0})')
plt.title('Convergence of Monte Carlo Fourier Estimation')
plt.legend()
plt.xscale('log')
plt.tight_layout()
plt.show()

print(f"\nAs N → ∞, estimate → exact value {exact:.4f}")
print("Error scales as O(1/√N) by Central Limit Theorem")

## 5. Influence Estimation

The **influence** of variable $i$ is:

$$\text{Inf}_i[f] = \Pr_x[f(x) \neq f(x^{\oplus i})]$$

where $x^{\oplus i}$ flips the $i$-th bit of $x$.

In [ ]:
# Estimate influences of MAJORITY₅
f = bf.majority(5)
exact_influences = f.influences()

print("Influence estimation for MAJORITY₅:")
print(f"{'Variable':<10} {'Exact':<12} {'Estimated':<12} {'Error':<10}")
print("-" * 45)

for i in range(5):
    exact = exact_influences[i]
    est, std_err = estimate_influence(f, i, n_samples=10000, return_confidence=True)
    error = abs(exact - est)
    print(f"Inf_{i}       {exact:<12.4f} {est:<12.4f} {error:<10.4f}")

print("\n→ All variables have equal influence (MAJORITY is symmetric)")

In [ ]:
# Total influence estimation
f = bf.majority(5)
exact_total = f.total_influence()
est_total = estimate_total_influence(f, n_samples=10000)

print(f"Total influence of MAJORITY₅:")
print(f"  Exact:     {exact_total:.4f}")
print(f"  Estimated: {est_total:.4f}")
print(f"  Error:     {abs(exact_total - est_total):.4f}")

## 6. The RandomVariableView Class

The `RandomVariableView` class provides a convenient interface for treating Boolean functions as random variables.

In [ ]:
# Create a random variable view
f = bf.tribes(2, 3)  # Tribes function
rv = RandomVariableView(f)

# Print summary
print(rv.summary())

In [ ]:
# Compare exact vs estimated values
f = bf.majority(5)
rv = RandomVariableView(f).seed(42)

print("Exact vs Estimated (n=10000 samples):")
print(f"  E[f]: exact={rv.expectation():.4f}, est={rv.estimate_expectation(10000):.4f}")
print(f"  Var[f]: exact={rv.variance():.4f}, est={rv.estimate_variance(10000):.4f}")
print(f"  I[f]: exact={rv.total_influence():.4f}, est={rv.estimate_total_influence(10000):.4f}")

In [ ]:
# Validate that estimates are close to exact values
f = bf.AND(4)
rv = RandomVariableView(f).seed(123)

results = rv.validate_estimates(n_samples=10000, tolerance=0.1)

print("Validation results (tolerance = 10%):")
for metric, passed in results.items():
    status = "✓" if passed else "✗"
    print(f"  {metric}: {status}")

## 7. Spectral Distribution Analysis

The `SpectralDistribution` class represents the Fourier weight distribution.

In [ ]:
# Analyze spectral distribution
f = bf.majority(5)
sd = SpectralDistribution.from_function(f)

print("Spectral Distribution of MAJORITY₅:")
print(f"  Shannon entropy: {sd.entropy():.4f} bits")
print(f"  Effective support (>1%): {sd.effective_support_size(0.01)} subsets")

print("\nWeight by degree:")
for k in range(6):
    w = sd.weight_at_degree(k)
    if w > 0.001:
        print(f"  W^{k}[f] = {w:.4f}")

In [ ]:
# Compare spectral distributions of different functions
functions = {
    "AND₄": bf.AND(4),
    "OR₄": bf.OR(4),
    "PARITY₄": bf.parity(4),
    "MAJORITY₅": bf.majority(5),
}

print(f"{'Function':<15} {'Entropy':<10} {'Eff. Support':<15}")
print("-" * 40)

for name, f in functions.items():
    sd = SpectralDistribution.from_function(f)
    print(f"{name:<15} {sd.entropy():<10.4f} {sd.effective_support_size(0.01):<15}")

## 8. P-Biased Analysis

The `RandomVariableView` also supports p-biased distributions.

In [ ]:
# Analyze AND under different biases
f = bf.AND(3)

print("AND₃ under different p-biases:")
print(f"{'p':<8} {'E[f]':<12} {'I[f]':<12}")
print("-" * 35)

for p in [0.1, 0.3, 0.5, 0.7, 0.9]:
    rv = RandomVariableView(f, p=p)
    print(f"{p:<8.1f} {rv.expectation():<12.4f} {rv.total_influence():<12.4f}")

print("\n→ E[AND] increases with p (more 1s → more likely all 1s)")
print("→ I[AND] peaks around p=0.5 (most sensitive to flips)")

## Summary

### Key Takeaways

1. **Boolean functions as random variables**: When inputs are drawn uniformly (or p-biased), $f(x)$ becomes a random variable with $\mathbb{E}[f] = \hat{f}(\emptyset)$

2. **Sampling**: `sample_uniform`, `sample_biased`, `sample_spectral` provide different ways to sample from the hypercube

3. **Monte Carlo estimation**: Fourier coefficients and influences can be estimated via sampling with $O(1/\sqrt{N})$ error

4. **Spectral distribution**: The distribution $\Pr[S] = \hat{f}(S)^2$ reveals the "frequency content" of $f$

5. **RandomVariableView**: A convenient class for probabilistic analysis of Boolean functions

### References

- O'Donnell, *Analysis of Boolean Functions*, Chapters 1-3
- Goldreich-Levin algorithm for finding heavy Fourier coefficients
- KM algorithm for learning juntas via spectral sampling

---

*This notebook demonstrates the `boofun.analysis.sampling` module.*